In [ ]:
from typing import Any, List, Tuple

import pandas as pd
import numpy as np

import xgboost as xgb
import catboost as cb

from sklearn.model_selection import StratifiedKFold

from lifelines.utils import concordance_index

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.auto import tqdm

In [ ]:
CAT_FEATURES = ["employee_count_nm", "bankemplstatus", "customer_age"]

In [ ]:
from objectives import CoxPHObjective, CoxPHMetric

In [ ]:
# from data_fusion_contest_2024_churn.ext.objectives import CoxPHObjective, CoxPHMetric

In [ ]:
def get_predict_k_fold(
    model: Any,
    X: pd.DataFrame,
    y: pd.Series,
    k_folds: int = 5,
    seed = 15
) -> Tuple[List[Any], List[pd.DataFrame], List[float]]:
    clfs = []
    scores = []
    preds_full = []
    kf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=seed)
    for train_index, test_index in tqdm(kf.split(X=X, y=y["target"])):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        model.fit(
            X=X_train, y=y_train[["label"]],
            eval_set=[(X_test, y_test[["label"]])],
            verbose=500
        )

        preds = model.predict(X_test)
        clfs += [model]
        scores.append(
            concordance_index(
                y_test["time"],
                preds,
                y_test["target"]
            )
        )

        preds_full.append(
            pd.DataFrame(
                {
                    "user_id": X.iloc[test_index].index,
                    "preds": -preds,
                    "target": y_test["target"].values,
                    "time": y_test["target"].values
                }
            )
        )
        print(scores)

    return clfs, preds_full, scores

## Чтение данных

In [ ]:
transactions = pd.read_csv(
    "../transactions.csv.zip", 
    parse_dates=["transaction_dttm"], 
    low_memory=False, compression="zip"
).sort_values(
    "transaction_dttm"
).reset_index(drop=True)

clients = pd.read_csv("../clients.csv")
report_dates = pd.read_csv("../report_dates.csv", parse_dates=["report_dt"])
train = pd.read_csv("../train.csv")

## Генерация фичей

In [ ]:
clients = clients.merge(
    report_dates, how="left", on="report"
)

In [ ]:
transactions = transactions.merge(
    clients, how="left", on="user_id"
)

In [ ]:
transactions["time"] = transactions["transaction_dttm"].apply(
    lambda x: x.hour * 3600 + x.minute * 60 + x.second
)

transactions["hour"] = transactions["transaction_dttm"].dt.hour

transactions["transaction_dttm"] = transactions["transaction_dttm"].dt.floor("D")

In [ ]:
count_trans = transactions.groupby(
    ["user_id"]
).agg(
        {
            "transaction_amt":[
                "sum",
                "min",
                "max",
                "median",
                "std",
                lambda x: np.percentile(x, 5),
                lambda x: np.percentile(x, 25),
                lambda x: np.percentile(x, 75),
                lambda x: np.percentile(x, 85),
                lambda x: np.percentile(x, 95)
            ],
            "mcc_code":[
                "nunique"
            ],
            "currency_rk":[
                "nunique"
            ],
            "transaction_dttm": [
                "min", "max", "nunique"
            ],
            "time": [
                "count",
                "max",
                "min",
                "mean",
                "median",
                "std"
            ]
        }
    )
count_trans.columns = count_trans.columns.map(
    "_".join
).map(lambda x: "count_trans_" + str(x))

count_trans["count_trans_dates"] = (
    count_trans["count_trans_transaction_dttm_max"] - count_trans["count_trans_transaction_dttm_min"]
).dt.days

count_trans.columns = count_trans.columns.str.replace('<', '').str.replace('>', '')

In [ ]:
transactions_hour_aggs = pd.pivot_table(
    data=transactions,
    index="user_id",
    values="transaction_amt",
    columns="hour",
    aggfunc=["count", "median"],
    fill_value=0
)
transactions_hour_aggs.columns = [
    f"hour_{col}" for col in transactions_hour_aggs.columns
]
hour_count_cols = [
        col for col in transactions_hour_aggs.columns
        if "count" in col
    ]
sum_ = transactions_hour_aggs[hour_count_cols].sum(axis=1)
for col in hour_count_cols:
    transactions_hour_aggs[col] = transactions_hour_aggs[col] / sum_

transactions_hour_aggs = transactions_hour_aggs.reset_index()

In [ ]:
transactions["trans_positive"] = np.where(
    transactions["transaction_amt"] > 0, transactions["transaction_amt"],np.nan
)
transactions["trans_negative"] = np.where(
    transactions["transaction_amt"] < 0, np.abs(transactions["transaction_amt"]),np.nan
)

In [ ]:
transactions["diff_days"] = (
    transactions["report_dt"] - transactions["transaction_dttm"]
) / np.timedelta64(1, "D")

nunique_days = transactions.groupby(
    "user_id", as_index=False
)["diff_days"].nunique().rename({"diff_days": "nunique_days"}, axis=1)

nunique_mcc_codes = transactions.groupby(
    "user_id", as_index=False
)["mcc_code"].nunique().rename({"mcc_code": "nunique_mcc_codes"}, axis=1)

nunique_currency = transactions.groupby(
    "user_id", as_index=False
)["currency_rk"].nunique().rename({"currency_rk": "nunique_currency"}, axis=1)

diff_days_aggs = transactions.groupby("user_id")["diff_days"].agg(
    ["min", "max", "mean", "std"]
)

diff_days_aggs.columns = [
    "diff_days_" + col for col in diff_days_aggs.columns
]

diff_days_aggs = diff_days_aggs.reset_index()

In [ ]:
transactions = transactions.merge(
    diff_days_aggs[["user_id", "diff_days_min"]], how="left", on="user_id"
)

In [ ]:
transactions["days_groups"] = 0
transactions.loc[
    transactions["diff_days"] <= transactions["diff_days_min"] + 10, "days_groups"
] = 10
transactions.loc[
    transactions["diff_days"] <= transactions["diff_days_min"] + 5, "days_groups"
] = 5
transactions.loc[
    transactions["diff_days"] == transactions["diff_days_min"], "days_groups"
] = 1

In [ ]:
trans_days_groups = transactions[~transactions["days_groups"].isin([0])].pivot_table(
    index = "user_id",
    values=["trans_positive", "trans_negative"],
    columns=["days_groups"],
    aggfunc=["count", "sum"]
)
trans_days_groups.columns = [
    f"days_groups_{x[0]}_{x[1]}_{x[2]}" for x in trans_days_groups.columns
]
trans_days_groups.reset_index(inplace=True)

In [ ]:
trans_cur_groups = transactions.pivot_table(
    index = "user_id",
    values=["trans_positive", "trans_negative"],
    columns=["currency_rk"],
    aggfunc=["count", "sum"]
)
trans_cur_groups.columns = [
    f"cur_groups_{x[0]}_{x[1]}_{x[2]}" for x in trans_cur_groups.columns
]
trans_cur_groups.reset_index(inplace=True)

In [ ]:
mcc_days_groups = transactions[~transactions["days_groups"].isin([0])].pivot_table(
    index = "user_id",
    values=["mcc_code"],
    columns=["days_groups"],
    aggfunc=["count", "nunique"]
)
mcc_days_groups.columns = [
    f"days_groups_mcc_{x[0]}_{x[1]}_{x[2]}" for x in mcc_days_groups.columns
]
mcc_days_groups.reset_index(inplace=True)

In [ ]:
%%time
transactions["date"] = transactions["transaction_dttm"].dt.date.astype("datetime64[ns]")
interval_trans = transactions.groupby("user_id").agg({"date":"unique"}).explode("date").reset_index()
interval_trans["interval"] = interval_trans.groupby("user_id")["date"].diff()
interval_trans = interval_trans.dropna() 
interval_trans["interval"] = interval_trans["interval"].dt.days
interval_trans = interval_trans.groupby(["user_id"]).agg({"interval": ["last","max"]})
interval_trans.columns = [f"date_{x[0]}_{x[1]}" for x in interval_trans.columns]
interval_trans.reset_index(inplace=True)

In [ ]:
count_mcc_code = transactions.mcc_code.value_counts().to_frame().reset_index().rename(
    columns={"index":"mcc_code", "count":"count_mcc_code"}
)

count_mcc_code20000 = np.array(
    count_mcc_code[
        (count_mcc_code["count_mcc_code"] > 20000)
        & (count_mcc_code["count_mcc_code"] < 1000000)
    ]["mcc_code"]
)

mcc_code_dumm20000 = pd.get_dummies(
    transactions[
        transactions["mcc_code"].isin(
            count_mcc_code20000
        )
    ].set_index("user_id")["mcc_code"]
)
mcc_code_dumm20000.columns = [f"mcc_count_{x}" for x in mcc_code_dumm20000.columns]
mcc_code_dumm20000 = mcc_code_dumm20000.groupby(["user_id"]).agg("sum").reset_index()

In [ ]:
count_mcc_code10000 = np.array(
    count_mcc_code[count_mcc_code["count_mcc_code"]>=1000000]["mcc_code"]
)
mcc_code_dumm10000 = transactions[transactions["mcc_code"].isin(count_mcc_code10000)][["user_id","mcc_code"]]
mcc_code_dumm10000 = mcc_code_dumm10000.groupby(["user_id"]).agg("count").reset_index().\
rename(columns={"mcc_code":"mcc_count_big"})

In [ ]:
count_mcc_code10000 = np.array(count_mcc_code[count_mcc_code["count_mcc_code"]<=20000]["mcc_code"])
mcc_code_dumm00001 = transactions[transactions["mcc_code"].isin(count_mcc_code10000)][["user_id","mcc_code"]]
mcc_code_dumm00001 = mcc_code_dumm00001.groupby(["user_id"]).agg("count").reset_index().\
rename(columns={"mcc_code":"mcc_count_small"})

In [ ]:
percent_last = clients[["user_id"]].copy()
for x in [3, 30, 60, 90]:
    prev = transactions[
        transactions["diff_days"] > x + 100
    ].groupby("user_id")["report_dt"].agg(
        ["count"]
    ).reset_index().rename(
        {
            "count": f"num_transaction_before_{x}_days"
        }, axis=1
    )
    last = transactions[
        transactions["diff_days"] <= x + 100
    ].groupby(
        "user_id"
    )["report_dt"].agg(
        ["count"]
    ).reset_index().rename(
        {"count": f"num_transaction_last_{x}_days"}, axis=1
    )

    percent_last = percent_last.merge(
        prev, how="left", on="user_id"
    )
    percent_last = percent_last.merge(
        last, how="left", on="user_id"
    )
    percent_last[f"num_transaction_last_{x}_days"].fillna(.000001, inplace=True)
    percent_last[f"num_transaction_before_{x}_days"].fillna(0.000001, inplace=True)
    
    percent_last[f"percent_last_{x}"] = (percent_last[f"num_transaction_last_{x}_days"] / \
    percent_last[f"num_transaction_before_{x}_days"]) * 100
    percent_last.drop(f"num_transaction_last_{x}_days", inplace=True, axis=1)
    percent_last.drop(f"num_transaction_before_{x}_days", inplace=True, axis=1)

In [ ]:
clients = pd.read_csv("../clients.csv")
df = clients.merge(
    train, on="user_id", how="left"
).merge(
    diff_days_aggs,
    on="user_id",
    how="left"
).merge(
    nunique_days, on="user_id", how="left"
).merge(
    trans_days_groups, on="user_id", how="left"
).merge(
    trans_cur_groups, on="user_id", how="left"
).merge(
    mcc_days_groups, on="user_id", how="left"
).merge(
    mcc_code_dumm20000, on="user_id", how="left"
).merge(
    mcc_code_dumm10000, on="user_id", how="left"
).merge(
    mcc_code_dumm00001, on="user_id", how="left"
).merge(
    percent_last, on="user_id", how="left",
).merge(
    count_trans, on="user_id", how="left",
).merge(
    transactions_hour_aggs, on="user_id", how="left",
).merge(
    nunique_mcc_codes, on="user_id", how="left",
).merge(
    nunique_currency, on="user_id", how="left",
)

df["pl_days_trans"] = (df["diff_days_max"] - df["diff_days_min"]) / df["nunique_days"]

In [ ]:
df.replace(
    {
        "employee_count_nm":{
            "ОТ 101 ДО 500": (100 + 500) // 2,
            "БОЛЕЕ 1001": 1001,
            "ОТ 501 ДО 1000": (501 + 1000) // 2,
            "ДО 10": 10 // 2,
            "ОТ 11 ДО 50": (11 + 50) // 2,
            "ОТ 51 ДО 100": (51 + 100) // 2,
            "БОЛЕЕ 500": 500,
            "ОТ 11 ДО 30": (11 + 30) // 2,
            "ОТ 31 ДО 50": (31 + 50) // 2
        }
    }, inplace=True
)

In [ ]:
df["label"] = np.where(df["target"]==0, -df["time"], df["time"])
df["time"] = df["time"].fillna(-1)
df["time"] = df["time"].astype(np.int32)
df["target"] = df["target"].fillna(-1)
df["target"] = df["target"].astype(np.int8)

In [ ]:
train_columns = [
    col for col in df.columns if col not in [
        "user_id",
        "report",
        "report_dt",
        "label",
        "target", 
        "time",
        "count_trans",
        "count_trans_transaction_dttm_min",
        "count_trans_transaction_dttm_max"
    ]
]

In [ ]:
df_train = df[(df["time"] != -1)].copy()

# Обучение моделей

In [ ]:
xgb_model = xgb.XGBRegressor(
    objective="survival:cox",
    random_state=15,
    reg_lambda=1.5,
    reg_alpha=1.4,
    subsample=0.8,
    colsample_bytree=0.3,
    gamma=3,
    min_child_weight=8,
    max_depth=10,
    learning_rate=0.005,
    n_estimators=4500
)

cb_model = cb.CatBoostRegressor(
    max_depth=6,
    subsample=0.8,
    colsample_bylevel=0.5,
    learning_rate=0.03,
    n_estimators=4500,
    loss_function=CoxPHObjective(),
    eval_metric=CoxPHMetric(),
    cat_features=CAT_FEATURES,
    use_best_model=True,
    random_state=15,
    thread_count=-1
)

In [ ]:
xgb_models, xgb_preds, xgb_scores = get_predict_k_fold(
    model=xgb_model,
    X=df_train[train_columns],
    y=df_train[["label", "target", "time"]],
    k_folds=5
)

In [ ]:
cb_models, cb_preds, cb_scores = get_predict_k_fold(
    model=cb_model,
    X=df_train.fillna(-999).astype(
        {
            "employee_count_nm": int
        }
    )[train_columns],
    y=df_train[["label", "target", "time"]],
    k_folds = 5,
    seed = 150
)

# Взвешенное усреднение Rank

In [ ]:
X_meta = pd.concat(
    cb_preds
).merge(
    pd.concat(xgb_preds)[["user_id", "preds"]],
    suffixes=("_cb", "_xgb"), on="user_id"
)

In [ ]:
X_meta["preds_cb"] = X_meta["preds_cb"].rank()
X_meta["preds_xgb"] = X_meta["preds_xgb"].rank()

In [ ]:
# TODO Стекинг

## Важность фичей

In [ ]:
def get_importance(
    features: pd.DataFrame,
    models: List[Any],
    height: float,
    top_n: int = 50
) -> pd.DataFrame:

    top_n = min(top_n, df.shape[1])

    fi = pd.DataFrame(index=features.columns, columns=[])
    for i, m in enumerate(models):
        fi[f"m_{i}"] = m.feature_importances_

    fi = fi.stack().reset_index().iloc[:, [0, 2]]
    fi.columns = ["feature", "importance"]

    cols_ord = (
        fi.groupby("feature")["importance"]
        .mean()
        .sort_values(ascending=False)
        .index.tolist()[:top_n]
    )

    fi = fi[fi["feature"].isin(cols_ord)]
    print(
        "Всего признаков {} Усреднее по {}-ти моделям: ".format(len(cols_ord), len(models))
    )

    plt.figure(figsize=(10, len(cols_ord) * height))
    _ = sns.boxplot(
        data=fi,
        y="feature",
        x="importance",
        orient="h",
        order=cols_ord
    )
    return (
        fi.groupby(by=["feature"], as_index=False)["importance"]
        .median()
        .sort_values(by="importance", ascending=False)
    )

In [ ]:
df_feats_imp = get_importance(
    df_train[train_columns],
    xgb_models,
    0.20,
    top_n=50,
)

# Прогноз

In [ ]:
X_pred = df[df["time"]==-1][train_columns].copy()

In [ ]:
xgb_predictions, cb_predictions = (
    np.zeros(len(X_pred)), np.zeros(len(X_pred))
)

for clf in xgb_models:
    xgb_predictions += clf.predict(X_pred[train_columns])

for clf in cb_models:
    cb_predictions += clf.predict(
        X_pred.fillna(-999).astype(
            {
                "employee_count_nm": int
            }
        )[train_columns]
    )

In [ ]:
submit = df[df["time"]==-1][["user_id"]].copy()
submit["predict"] = pd.Series(xgb_predictions).rank().values * 0.6 + pd.Series(cb_predictions).rank().values * 0.4
submit.to_csv(f"submission_blend.csv",index=False)
submit